Dummy

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


import sys

sys.path.append('../src')

from data_preprocessing import df,  y

In [ ]:
# Definiera X, y och cv
import sklearn.model_selection as skl_ms

X = df.drop("increase_stock", axis=1)
cv = skl_ms.StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

In [37]:
day_cols = [c for c in X.columns if c.startswith("day_")]
binary_cols = ["snow_or_not", "summertime"]
cols_to_scale = [c for c in X.columns if c not in day_cols + binary_cols]


preprocessor = ColumnTransformer(
    transformers=[("scale", StandardScaler(), cols_to_scale)],
    remainder="passthrough",
)


dummy_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", DummyClassifier(strategy='stratified', random_state=1))
])


baseline_score = cross_val_score(dummy_pipeline, X, y, cv=cv, scoring='accuracy').mean()

print(f"Baseline (Dummy) Accuracy: {baseline_score:.4f}")

Baseline (Dummy) Accuracy: 0.6913


In [38]:
strategies = ['stratified', 'most_frequent', 'uniform']
for strategy in strategies:
    dummy = DummyClassifier(strategy=strategy, random_state=1)
    pipeline = Pipeline([("preprocess", preprocessor), ("model", dummy)])
    accuracy = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy').mean()
    f1 = cross_val_score(pipeline, X, y, cv=cv, scoring='f1_weighted').mean()
    
    print(f"{strategy:15} | Accuracy: {accuracy:.4f} | F1-score: {f1:.4f}")

stratified      | Accuracy: 0.6913 | F1-score: 0.6937
most_frequent   | Accuracy: 0.8200 | F1-score: 0.7389
uniform         | Accuracy: 0.5287 | F1-score: 0.5833
